In [2]:
!pip3 install deepspeech

In [1]:
import deepspeech
model_file_path = 'deepspeech-0.9.3-models.pbmm'
beam_width = 500
model = deepspeech.Model(model_file_path, beam_width)


In [1]:
import wave
filename = 'example.wav'
w = wave.open(filename, 'r')
rate = w.getframerate()
frames = w.getnframes()
buffer = w.readframes(frames)
print(rate)
print(model.sampleRate())
type(buffer)

16000


NameError: name 'model' is not defined

In [ ]:
import numpy as np
data16 = np.frombuffer(buffer, dtype=np.int16)
type(data16)

In [ ]:
text = model.stt(data16)
print(text)